# 📱 Lecture 13: On-Device Training - Complete Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gaurav-redhat/efficientml_course/blob/main/13_on_device_training/demo.ipynb)

## What You'll Learn
- Why train on edge devices
- TinyTL: Lite training
- Sparse updates and bias-only training
- Memory-efficient backpropagation

In [ ]:
!pip install torch matplotlib numpy -q
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(42)
print('Ready for On-Device Training!')

## Part 1: Why On-Device Training?

- **Privacy**: Data never leaves device
- **Personalization**: Adapt to user's data
- **Latency**: No network round-trip
- **Offline**: Works without connectivity

In [ ]:
# Constraints comparison
platforms = {
    'Cloud GPU (A100)': {'memory_gb': 80, 'compute_tflops': 312},
    'Laptop GPU (4060)': {'memory_gb': 8, 'compute_tflops': 15},
    'Smartphone (A16)': {'memory_gb': 6, 'compute_tflops': 2},
    'MCU (STM32H7)': {'memory_gb': 0.001, 'compute_tflops': 0.0001},
}

print('📊 PLATFORM COMPARISON')
print('=' * 60)
print(f'{"Platform":<25} {"Memory":<15} {"Compute (TFLOPS)":<15}')
print('-' * 60)
for name, specs in platforms.items():
    mem = f'{specs["memory_gb"]:.3f} GB' if specs['memory_gb'] < 1 else f'{specs["memory_gb"]:.0f} GB'
    print(f'{name:<25} {mem:<15} {specs["compute_tflops"]:<15.4f}')

print('\n⚠️ On-device training needs 1000x less memory!')

## Part 2: Full Fine-tuning vs Lite Training

In [ ]:
class MobileNetBlock(nn.Module):
    """Simplified MobileNet block."""
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, in_ch, 3, stride, 1, groups=in_ch, bias=False)
        self.bn1 = nn.BatchNorm2d(in_ch)
        self.conv2 = nn.Conv2d(in_ch, out_ch, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_ch)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        return x

class TinyTLModel(nn.Module):
    """
    Model supporting different training modes:
    1. Full: Train all parameters
    2. Last layer: Only train classifier
    3. Bias-only (TinyTL): Only train biases
    4. Lite residual: Train small adapter modules
    """
    def __init__(self, num_classes=10):
        super().__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, 2, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            MobileNetBlock(32, 64, stride=2),
            MobileNetBlock(64, 128, stride=2),
            MobileNetBlock(128, 256, stride=2),
        )
        
        # Lite residual adapters (small, trainable)
        self.adapters = nn.ModuleList([
            nn.Conv2d(64, 64, 1, bias=True),
            nn.Conv2d(128, 128, 1, bias=True),
            nn.Conv2d(256, 256, 1, bias=True),
        ])
        
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(256, num_classes)
    
    def forward(self, x):
        x = self.features(x)
        x = self.gap(x).flatten(1)
        return self.classifier(x)
    
    def set_training_mode(self, mode='full'):
        """
        Set which parameters to train.
        
        Modes:
        - full: All parameters
        - last_layer: Only classifier
        - bias_only: Only bias terms
        - lite_residual: Only adapters + classifier
        """
        # First freeze everything
        for param in self.parameters():
            param.requires_grad = False
        
        if mode == 'full':
            for param in self.parameters():
                param.requires_grad = True
        
        elif mode == 'last_layer':
            for param in self.classifier.parameters():
                param.requires_grad = True
        
        elif mode == 'bias_only':
            for name, param in self.named_parameters():
                if 'bias' in name:
                    param.requires_grad = True
            # Also train classifier
            for param in self.classifier.parameters():
                param.requires_grad = True
        
        elif mode == 'lite_residual':
            for param in self.adapters.parameters():
                param.requires_grad = True
            for param in self.classifier.parameters():
                param.requires_grad = True
        
        return self.count_trainable_params()
    
    def count_trainable_params(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

# Create model and compare modes
model = TinyTLModel()
total_params = sum(p.numel() for p in model.parameters())

print('📊 TRAINING MODE COMPARISON')
print('=' * 60)
print(f'{"Mode":<20} {"Trainable Params":<20} {"% of Total":<15}')
print('-' * 60)

modes = ['full', 'last_layer', 'bias_only', 'lite_residual']
results = []

for mode in modes:
    trainable = model.set_training_mode(mode)
    pct = trainable / total_params * 100
    results.append((mode, trainable, pct))
    print(f'{mode:<20} {trainable:>15,}   {pct:>12.2f}%')

print(f'\nTotal parameters: {total_params:,}')

In [ ]:
# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Trainable parameters
modes = [r[0] for r in results]
trainable = [r[1] for r in results]
pcts = [r[2] for r in results]

colors = ['#ef4444', '#f59e0b', '#22c55e', '#3b82f6']
axes[0].bar(modes, trainable, color=colors)
axes[0].set_ylabel('Trainable Parameters')
axes[0].set_title('Trainable Parameters by Mode')
axes[0].set_yscale('log')

for i, (t, p) in enumerate(zip(trainable, pcts)):
    axes[0].text(i, t * 1.5, f'{p:.1f}%', ha='center')

# Memory savings (simplified estimate)
# Training memory ≈ params × 4 (gradients) + activations
base_mem = 100  # Arbitrary baseline
mem_savings = [base_mem, base_mem * 0.3, base_mem * 0.1, base_mem * 0.15]

axes[1].bar(modes, mem_savings, color=colors)
axes[1].set_ylabel('Training Memory (relative)')
axes[1].set_title('Training Memory by Mode')

plt.tight_layout()
plt.show()

## Part 3: Memory-Efficient Backpropagation

In [ ]:
def backprop_memory_analysis():
    """
    Analyze memory requirements for different training strategies.
    """
    print('📊 BACKPROP MEMORY ANALYSIS')
    print('=' * 60)
    print('\nFull backprop needs to store:')
    print('  1. All intermediate activations')
    print('  2. All gradients')
    print('  3. Optimizer states')
    
    print('\nBias-only training:')
    print('  1. Only bias-related activations (much smaller)')
    print('  2. Only bias gradients')
    print('  3. Minimal optimizer states')
    
    # Simplified memory calculation
    # Assume: 10-layer network, each layer has 1M params, 10KB bias
    n_layers = 10
    params_per_layer = 1_000_000
    bias_per_layer = 10_000
    
    # Full training
    full_grad_mem = n_layers * params_per_layer * 4  # FP32 gradients
    full_act_mem = n_layers * params_per_layer * 4   # Activations
    
    # Bias-only
    bias_grad_mem = n_layers * bias_per_layer * 4
    bias_act_mem = n_layers * bias_per_layer * 4
    
    print(f'\n📊 MEMORY COMPARISON')
    print(f'{"Component":<20} {"Full (MB)":<15} {"Bias-only (MB)":<15} {"Savings":<10}')
    print('-' * 60)
    print(f'{"Gradients":<20} {full_grad_mem/1e6:<15.1f} {bias_grad_mem/1e6:<15.3f} {full_grad_mem/bias_grad_mem:<10.0f}x')
    print(f'{"Activations":<20} {full_act_mem/1e6:<15.1f} {bias_act_mem/1e6:<15.3f} {full_act_mem/bias_act_mem:<10.0f}x')

backprop_memory_analysis()

## Part 4: Training Comparison Experiment

In [ ]:
# Simulate training with different modes
def train_model(model, mode, X_train, y_train, X_test, y_test, epochs=20):
    model.set_training_mode(mode)
    
    optimizer = torch.optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=0.001
    )
    criterion = nn.CrossEntropyLoss()
    
    history = {'train_loss': [], 'test_acc': []}
    
    for epoch in range(epochs):
        # Train
        model.train()
        optimizer.zero_grad()
        out = model(X_train)
        loss = criterion(out, y_train)
        loss.backward()
        optimizer.step()
        
        # Evaluate
        model.eval()
        with torch.no_grad():
            test_out = model(X_test)
            acc = (test_out.argmax(1) == y_test).float().mean().item() * 100
        
        history['train_loss'].append(loss.item())
        history['test_acc'].append(acc)
    
    return history

# Create data
X_train = torch.randn(200, 3, 64, 64)
y_train = torch.randint(0, 10, (200,))
X_test = torch.randn(50, 3, 64, 64)
y_test = torch.randint(0, 10, (50,))

print('📚 TRAINING EXPERIMENT')
print('=' * 50)

all_histories = {}

for mode in ['full', 'last_layer', 'bias_only', 'lite_residual']:
    print(f'\nTraining with {mode} mode...')
    model = TinyTLModel()
    history = train_model(model, mode, X_train, y_train, X_test, y_test, epochs=30)
    all_histories[mode] = history
    print(f'  Final accuracy: {history["test_acc"][-1]:.1f}%')

In [ ]:
# Visualize training results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = {'full': '#ef4444', 'last_layer': '#f59e0b', 'bias_only': '#22c55e', 'lite_residual': '#3b82f6'}

for mode, history in all_histories.items():
    axes[0].plot(history['train_loss'], label=mode, color=colors[mode], linewidth=2)
    axes[1].plot(history['test_acc'], label=mode, color=colors[mode], linewidth=2)

axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Training Loss')
axes[0].set_title('Training Loss by Mode')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Test Accuracy (%)')
axes[1].set_title('Test Accuracy by Mode')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary
print('\n📊 FINAL SUMMARY')
print('=' * 60)
print(f'{"Mode":<15} {"Trainable %":<15} {"Final Acc":<15} {"Memory":<15}')
print('-' * 60)

mem_estimate = {'full': '100%', 'last_layer': '30%', 'bias_only': '10%', 'lite_residual': '15%'}

for mode, history in all_histories.items():
    trainable_pct = [r[2] for r in results if r[0] == mode][0]
    print(f'{mode:<15} {trainable_pct:>12.1f}%  {history["test_acc"][-1]:>12.1f}%  {mem_estimate[mode]:>12}')

In [ ]:
print('🎯 KEY TAKEAWAYS')
print('=' * 60)
print('\n1. On-device training enables privacy and personalization')
print('\n2. Full fine-tuning is too expensive for edge devices')
print('\n3. Bias-only (TinyTL): 100x less memory, good accuracy')
print('\n4. Lite residual: Small adapters for better adaptation')
print('\n5. Last-layer only: Simplest but limited')
print('\n6. Choose based on memory budget and accuracy needs')
print('\n' + '=' * 60)
print('\n📚 Next: Distributed Training!')